In [1]:
import json
from sklearn.utils import shuffle
import re
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from deeppavlov import build_model
from deeppavlov import train_model, configs
from deeppavlov.core.commands.utils import parse_config
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import pipeline
import torch
from transformers import BertForQuestionAnswering

import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

d:\Jupiter\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('dataset\\train.json', 'r', encoding='utf-8') as f: #открыли файл с данными
    data = json.load(f) #загнали все, что получилось в переменную
data[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

### Задачу можно свести к типу Question Answering, где вопросом будет метка, а ответом нужный кусок текста 

#### Приведём данные к виду 
id (int), title (str), context (str), question (str), answers (dict) {'text': кусок текста, "answer_start": [Начало индекса]}

In [3]:
def preprocessing(data):
    max_lenght_ques = 0
    max_lenght_answ = 0
    data_train = []
    question_id = {'обеспечение исполнения контракта': 0, 'обеспечение гарантийных обязательств': 1}

    for dict_data in data:
        id =  int(dict_data.get('id')) # id 
        text = dict_data.get('text') # Исходный текст
        question = dict_data.get('label')   # Метка, которую мы будем рассматривать как вопрос 
        answer_start = int(dict_data.get('extracted_part').get('answer_start')[0]) # Индекс начала ответа
        answer_end = dict_data.get('extracted_part').get('answer_end')[0] # Индекс конца ответа

        
        if (answer_start != 0 or answer_end != 0):
            if max_lenght_answ < answer_end - answer_start:
                max_lenght_answ = answer_end - answer_start

            answers = {'answer_start': answer_start, 'text': dict_data.get('extracted_part').get('text')[0]}
            #row = {"context": text, 'qas': [{"id":id,"is_impossible": False,"question": question,"answers": [answers]}]}
            row = {"context": text,"id":id, 'qas': [{"id":id, "question": question,"answers": [answers]}]}
            #data_train.append(row)
            data_train.append(row)
        else:
            pass

    print(max_lenght_answ)

    
    data_train, data_valid = train_test_split(data_train, train_size=0.75, random_state=12345)
    print('Размер тренировочной выборки: ',len(data_train))
    print('Размер валидационной выборки: ',len(data_valid))

    data_train = {"version": 1.1, "data": [{"title": "KonturContext", "paragraphs": data_train}]}
    data_valid = {"version": 1.1, "data": [{"title": "KonturContext", "paragraphs": data_valid}]}
    
    return data_train, data_valid

data_train, data_valid = preprocessing(data)

data_train.get('data')[0].get('paragraphs')[0]

302
Размер тренировочной выборки:  1119
Размер валидационной выборки:  373


{'context': '35 УТВЕРЖДАЮ Ломакина Наталья Владимировна Директор МБУ "МФЦ городского округа Балашиха" «01» сентября 2022г. ОРМЕ» документации, а именно: 1 час с момента размещения оператором электронной площадки протокола сопоставления ценовых предложений. 26. Дата начала и окончания срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме Дата начала срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме: «12» сентября 2022 Дата окончания срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме: «12» сентября 2022 27. Дата подведения итогов аукциона в электронной форме Дата подведения итогов аукциона в электронной форме: «12» сентября 2022 28. Обеспечение заявок на участие в аукционе в электронной форме Не требуется 28.1. Размер обеспечения заявок на участие в аукционе в электронной форме Не требуется 28.2. Срок и порядок предоставления обеспечения заявок на участие в аукционе в электронной форме Не требуется

In [41]:
with open('dataset\\train-v1.1.json', 'w', encoding='utf-8') as outfile:
    json.dump(data_train, outfile, ensure_ascii=False)

with open('dataset\\dev-v1.1.json', 'w', encoding='utf-8') as outfile:
    json.dump(data_valid, outfile, ensure_ascii=False)

In [32]:
from datasets import load_dataset

raw_datasets = load_dataset("sberquad")
print(raw_datasets['train'][0])
del raw_datasets

Found cached dataset sberquad (C:/Users/ykolt/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77)
100%|██████████| 3/3 [00:00<00:00, 184.76it/s]

{'id': 62310, 'title': 'SberChallenge', 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.', 'question': 'чем представлены органические остатки?', 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'], 'answer_start': [109]}}


In [ ]:
import gc
gc.collect()

4

In [20]:
model_config = parse_config('squad_ru_convers_distilrubert_2L')

In [21]:
print(model_config['dataset_reader']['data_path'])
print(model_config['train']['batch_size'])

~/.deeppavlov/downloads/squad_ru_clean/
10


In [22]:
model_config['dataset_reader']['data_path'] = 'D:\Jupiter\Kontur\dataset'
#model_config['train']['batch_size'] = 4

In [23]:
print(model_config['dataset_reader']['data_path'])
print(model_config['train']['batch_size'])

D:\Jupiter\Kontur\dataset
10


In [62]:
import gc
gc.collect()

6564

In [4]:
model = build_model('squad_ru_convers_distilrubert_6L', download=False, install=False)

Some weights of the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational and are newly initialized: ['qa_outputs.bias', 'qa_

In [56]:
data_train.get('data')[0].get('paragraphs')[1]

{'context': 'КОНТРАКТ<1> № ___ <2> на поставку продуктов питания (бакалея 4) (Идентификационный код закупки -222711101260771110100101240001061244 ) ________<4> «__» _______ 20__<5> ГУЗ «Алексинская районная больница № 1 имени профессора В.Ф. Снегирева» <6>, отребовать уплату штрафа. Размер штрафа определяется в соответствии с Правилами и составляет 1000 (одна тысяча) рублей 00 копеек. 7.14. Применение неустойки (штрафа, пени) не освобождает Стороны от исполнения обязательств по настоящему Контракту. 7.15. Общая сумма начисленных штрафов за неисполнение или ненадлежащее исполнение Поставщиком обязательств, предусмотренных настоящим Контрактом, не может превышать цену Контракта. 7.16. Общая сумма начисленных штрафов за ненадлежащее исполнение Заказчиком обязательств, предусмотренных настоящим Контрактом, не может превышать цену Контракта. 7.17. В случае расторжения настоящего Контракта в связи с односторонним отказом Стороны от исполнения настоящего Контракта другая Сторона вправе потреб

In [6]:
def accuracy(data):
    count = len(data)
    count_true = 0
    for i in range(len(data)):  
        text = data[i].get('context')
        quest = data[i].get('qas')[0].get('question')
        answ = data[i].get('qas')[0].get('answers')[0].get('text')

        predict = model([text], [quest])[0][0]
        
        if predict == answ:
            count_true+=1
    
    return count_true/count

print(accuracy(data_valid.get('data')[0].get('paragraphs')))

print(accuracy(data_train.get('data')[0].get('paragraphs')))

0.3324396782841823
0.44682752457551383


In [49]:
i = 6
text = data[i].get('text')
quest = data[i].get('label')

print(model([text], [quest]))
print(data[i].get('extracted_part').get('text'))

[['Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта'], [1298], [1.0]]
['Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта']


In [4]:
import torch
  
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"torch version: {torch.__version__}")

Is CUDA supported by this system? True
CUDA version: 11.8
torch version: 2.0.0+cu118
